# A hypothetical case study for stochastic optimization: the classic farmer problem

A farmer wants to plant three types of crops: Wheat, Corn, and Beans. He has a fixed amount of land and aims to meet the demand for each type of crop in the local market. The demand for each crop is uncertain. The yield of each crop is uncertain and depends on the amount of water they receive during the season (endogenous uncertainty). Additionally, the market prices for each crop at the end of the season are uncertain (exogenous uncertainty).

The farmer can choose to invest in an irrigation system (with three possible operating levels: Low, Medium, High) that influences the amount of water each crop receives. The investment and operational costs vary with the level.

### Nomenclature:

### Set:

- $i \in \in \{1, 2, 3\}$: Category of produce, Wheat, Corn, and Beans respectively.

- $j \in \{1, 2, 3\}$: Irrigation operating level(low, medium, and high respectively).

#### Decision Variables:

- $x_{i,j}$: Acres of land planted with produce i under irrigation operating level j.

- $y_{i,j}$ ($j \in \{1, 2, 3\}$): Binary, decision for produce i under irrigation operating level j.

#### Intermediate variables:

- $z_i$: Wheat, Corn and Beans product

#### Uncertain Parameters:

- $\xi_{i, j}$: Yields for produce i with irrigation operating level j(endogenous uncertainty).
- $P_i$: Market prices for Wheat, Corn, and Beans at the end of the season (exogenous uncertainty).
- $D_i$: Market demand for Wheat, Corn, and Beans (exogenous uncertainty).

#### Probability distribution of uncertain parameters(when realized in scenario s):

- $\Psi^{\xi}_{i,j,s}, \Psi^{P}_{i,s}, \Psi^{D}_{i,s}$ Probability of uncertain parameters

#### Deterministic parameters

- $R_{i}$ unit cost of seeds for Wheat, Corn, and Beans, respectively.

- $C^{f}_{i}$ fixed cost of irrigation operating level i.

- $C^{v}_{i, }$ variable cost of irrigation operating level i.

- $L$: total land available.

### Deterministic Formulation

#### Constraints:

1. **Total Land Constraint**:
\begin{align*}
\sum_{i,j} x_{i,j} \le L
\end{align*}

2. **Demand Constraint**: The production of each crop should not exceed its respective market demand.
\begin{align*}
z_i &\le D_i \\
\end{align*}

3. **Irrigation Choice Constraints I**:
\begin{align*}
\sum_{j} y_{i,j} = 1 \quad \forall i
\end{align*}

4. **Irrigation Choice Constraints II**:
\begin{align*}
x_{i,j} \le Ly_{i,j} \quad \forall i,j
\end{align*}

5. **Yield Constraints**:

\begin{align*}
\sum_{j} \xi_{i,j} x_{i,j} = z_i
\end{align*}

#### Objective:

$$ \textbf{min} \quad \sum_{i}  \{R_{i}\sum_{j} {x_{i,j}} + \sum_{j} C^{f}_{i,j} {y_{i,j}} + \sum_{j} C^{v}_{i,j} {x_{i,j}} \} - \sum_{i} D_{i} {z_{i}}$$

### Stochastic formulation given scenarios S

#### Constraints:

1. **Total Land Constraint**:
\begin{align*}
\sum_{i,j} x_{i,j} \le L
\end{align*}

2. **Demand Constraint**: The production of each crop should not exceed its respective market demand.
\begin{align*}
z_i &\le D_{i,s} \forall s\\
\end{align*}

3. **Irrigation Choice Constraints I**:
\begin{align*}
\sum_{j} y_{i,j} = 1 \quad \forall i
\end{align*}

4. **Irrigation Choice Constraints II**:
\begin{align*}
x_{i,j} \le Ly_{i,j} \quad \forall i,j
\end{align*}

5. **Yield Constraints**:

\begin{align*}
\sum_{j} \xi_{i,j,s} x_{i,j} = z_{i,s}
\end{align*}

#### Objective:

$$ \textbf{min} \quad \sum_{i}  \{R_{i}\sum_{j} {x_{i,j}} + \sum_{j} C^{f}_{i,j} {y_{i,j}} + \sum_{j} C^{v}_{i,j} {x_{i,j}} \} - \{ \sum_{s} \sum_{i} D_{i} {z_{i}} \}$$